In [1]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 40.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
import gdown
import json
import random
from difflib import get_close_matches
import spacy
nlp = spacy.load("en_core_web_sm")
file_id = '1WD4TdDzfBQq5buxg7P2i-zqcRtg4bVyK'
url = f'https://drive.google.com/uc?id={file_id}'
output = 'songs.json'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1WD4TdDzfBQq5buxg7P2i-zqcRtg4bVyK
To: /content/songs.json
100%|██████████| 20.6k/20.6k [00:00<00:00, 39.9MB/s]


'songs.json'

In [16]:
emotion_keywords = {
    "sadness": ["sad", "blue", "depressed", "melancholy", "down", "gloomy", "heartbroken", "despair", "desperate", "upset"],
    "anger": ["angry", "furious", "irritated", "annoyed", "resentful", "rage", "outraged", "fuming"],
    "fear": ["afraid", "scared", "fearful", "anxious", "worried", "terrified", "panicked", "nervous"],
    "disgust": ["disgusted", "repulsed", "bored", "nauseous", "revolted", "sickened", "contempt"],
    "surprised" : ["surprised", "shocked", "amazed", "astonished", "astounded", "stunned"],
    "loneliness": ["lonely", "isolated", "abandoned", "alone", "forlorn", "lonesome"],
    "happiness": ["happy", "joyful", "cheerful", "delighted", 'fine'],
    "hope": ["hopeful", "optimistic", "encouraged", "expectant", "confident"],
    "disappointment": ["disappointed", "let down", "crestfallen", "disheartened", "dissatisfied"],
    "brokenness": ["broken", "shattered", "crushed", "defeated", "devastated", "ruined"],
    "envy": ["envious", "jealous", "covetous", "resentful", "green-eyed", "begrudging"],
    "suspicion": ["suspicious", "distrustful", "doubtful", "mistrustful", "wary", "skeptical"],
    "fault": ["guilty", "blameworthy", "culpable", "responsible", "accountable", "at fault"],
    "shame": ["ashamed", "humiliated", "embarrassed", "mortified", "disgraced", "demeaned"],
    "apathy": ["apathetic", "indifferent", "uninterested", "unconcerned", "detached", "numb"],
    "detachment": ["detached", "disconnected", "removed", "isolated", "separated", "aloof"],
    "emptiness": ["empty", "hollow", "void", "unfulfilled", "vacant", "blank"],
    "being lost": ["lost", "confused", "directionless", "adrift", "aimless", "bewildered"],
    "weakness": ["weak", "powerless", "helpless", "feeble", "frail", "vulnerable"],
    "stupor": ["stunned", "dazed", "numb", "shocked", "bewildered", "confused"],
    "chagrin": ["chagrined", "annoyed", "vexed", "irritated", "displeased", "frustrated"],
    "despair": ["hopeless", "despondent", "discouraged", "defeated", "wretched", "forlorn"],
    "shock": ["shocked", "traumatized", "stunned", "dumbfounded", "flabbergasted", "appalled"],
    "hopelessness": ["hopeless", "pessimistic", "defeatist", "gloomy", "bleak", "dreary"],
    "frustration": ["frustrated", "exasperated", "aggravated", "irked", "discontented", "annoyed"],
    "joy": ["happy", "joyful", "delighted", "ecstatic", "elated"],
    "love": ["loving", "affectionate", "fond", "caring", "adoring", "devoted"],
    "peace": ["peaceful", "calm", "serene", "tranquil", "placid", "composed"],
    "contentment": ["content", "satisfied", "fulfilled", "gratified", "pleased", "happy"],
    "excitement": ["excited", "thrilled", "eager", "enthusiastic", "animated", "pumped", 'energetic'],
    "pride": ["proud", "satisfied", "dignified", "self-respecting", "confident", "pleased"],
    "gratitude": ["grateful", "thankful", "appreciative", "obliged", "indebted", "moved"],
    "relief": ["relieved", "reassured", "comforted", "unburdened", "lightened", "eased"],
    "amusement": ["amused", "entertained", "delighted", "charmed", "tickled", "pleased"],
    "awe": ["awed", "wonderstruck", "amazed", "astonished", "reverent", "impressed"],
    "confidence": ["confident", "assured", "self-assured", "poised", "self-confident", "secure"],
    "courage": ["brave", "courageous", "bold", "fearless", "valiant", "heroic"],
    "determination": ["determined", "resolute", "decided", "purposeful", "committed", "persistent"],
    "inspiration": ["inspired", "motivated", "stimulated", "spurred", "encouraged", "influenced"]
}

keyword_to_emotion = {}
for emotion, keywords in emotion_keywords.items():
    for kw in keywords:
        keyword_to_emotion[kw] = emotion
with open('songs.json', 'r', encoding='utf-8') as f:
    songs_db = json.load(f)
questions = {
    "who_are_you": ["who are you", "what's your name"],
    "what_can_you_do": ["what can you do", "how can you help", "your purpose"]
}

greetings_phrases = ["hi", "hello", "hey", "greetings"]
farewell_phrases = ["bye", "goodbye", "see you", "exit", "quit"]
thanks_phrases = ["thanks", "thank you", "thx", 'great', 'that`s great', 'that is great']
how_are_you_phrases = ["how are you", "how's it going"]
good_day_phrases = ["good morning", "good evening", 'good afternoon', 'good night']
whats_up_phrases = ["what's up", "sup"]
identity_phrases = ["who are you", "what are you", 'are you a chat bot', 'are you chat bot', 'are you AI', 'are you ai', 'are you human', 'are you a human']
capability_phrases = ["what can you do", "help me", "your purpose"]
offtopic_phrases = [
    "weather", "time", "news", "sports", "politics",
    "movie", "book", "game", "food", "travel",
    "tell me about", "what about", "do you know"
]

offtopic_responses = [
    "I can't help with that, but here's a song while you figure it out:",
    "That's outside my expertise, but maybe this music will help:",
    "I'm just a music bot, but I can suggest this track:",
    "For that you might need a different assistant, but enjoy this song:"
]

compliments = [
    "That's wonderful to hear!",
    "You're amazing!",
    "Keep spreading that positive energy!",
    "Your positivity is inspiring!",
    "That's fantastic! Keep it up!"
]
additional_phrases = ["wow", "i like it"]

def get_song_for_emotion(emotion):
    if emotion in songs_db and songs_db[emotion]:
        song = random.choice(songs_db[emotion])
        return {
            "message": song["description"],
            "song": {
                "title": song["title"],
                "artist": song["artist"],
                "url": song["link"]
            }
        }
    else:
        all_songs = []
        for s_list in songs_db.values():
            all_songs.extend(s_list)
        if all_songs:
            song = random.choice(all_songs)
            return {
                "message": "Here's a song that might fit your mood:",
                "song": {
                    "title": song["title"],
                    "artist": song["artist"],
                    "url": song["link"]
                }
            }
    return None
def handle_offtopic_request():
    response = random.choice(offtopic_responses)
    song_data = get_song_for_emotion(random.choice(list(songs_db.keys())))
    if song_data:
        print(f"Bot: {response}")
        print(f"'{song_data['song']['title']}' by {song_data['song']['artist']}")
        print(f"Description: {song_data['message']}")
        print(f"Listen here: {song_data['song']['url']}")
    else:
        print("Bot: I can't help with that topic. Maybe try describing your mood?")

def process_input(text):
    doc = nlp(text.lower())
    lemmas = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    words = [token.text for token in doc if not token.is_stop and not token.is_punct]
    return lemmas + words
def identify_emotion_or_request(text):
    lower_text = text.lower()
    processed = process_input(text)
    positive_emotions = ["hope", "joy", "love", "peace", "contentment", "excitement", "pride", "gratitude", "relief", "amusement", "awe", "confidence", "courage", "determination", "inspiration"]
    if any(phrase in lower_text for phrase in offtopic_phrases):
      return {"type": "offtopic"}

    for word in processed:
        if word in keyword_to_emotion:
            emotion = keyword_to_emotion[word]
            if emotion in positive_emotions:
                return {"type": "positive_emotion", "value": emotion, "immediate": True}
            else:
                return {"type": "negative_emotion", "value": emotion, "immediate": True}

    # Pattern: "[someone] is [emotion]"
    doc = nlp(text.lower())
    for token in doc:
        if token.dep_ == "ROOT" and token.pos_ == "VERB" and token.text in ["is", "are", "feel", "feels"]:
            for child in token.children:
                if child.dep_ == "acomp":
                    adj = child.text
                    if adj in keyword_to_emotion:
                        emotion = keyword_to_emotion[adj]
                        if emotion in positive_emotions:
                            return {"type": "positive_emotion", "value": emotion, "immediate": True}
                        else:
                            return {"type": "negative_emotion", "value": emotion, "immediate": True}


    for i, token in enumerate(doc[:-1]):
        if token.text == "feeling" and doc[i+1].text in keyword_to_emotion:
            emotion = keyword_to_emotion[doc[i+1].text]
            if emotion in positive_emotions:
                return {"type": "positive_emotion", "value": emotion, "immediate": True}
            else:
                return {"type": "negative_emotion", "value": emotion, "immediate": True}


    all_keywords = list(keyword_to_emotion.keys())
    for word in processed:
        matches = get_close_matches(word, all_keywords, n=1, cutoff=0.7)
        if matches:
            emotion = keyword_to_emotion[matches[0]]
            if emotion in positive_emotions:
                return {"type": "positive_emotion", "value": emotion}
            else:
                return {"type": "negative_emotion", "value": emotion}

    for key, phrases in questions.items():
        if any(phrase in lower_text for phrase in phrases):
            return {"type": "question", "value": key}


    return {"type": "unknown"}

def handle_clarification_response(response, last_emotion):
    if not last_emotion:
        print("Bot: I'm not sure what you mean. Could you tell me more about how you're feeling?")
        return False
    response_lower = response.lower()
    if any(w in response_lower for w in ["better", "cheer", "happy", "lift", "up", "improve", "change"]):
        song_data = get_song_for_emotion("hope")
        if song_data:
            print(f"Bot: {song_data['message']}")
            print(f"Bot: I'm sending positive vibes your way! Try this uplifting song: '{song_data['song']['title']}' by {song_data['song']['artist']}")
            print(f"Listen here: {song_data['song']['url']}")
        else:
            print("Bot: I couldn't find a suitable song, but I hope you feel better soon!")
        return True
    elif any(w in response_lower for w in ["match", "current", "same", "keep", "stay", "continue", "sad", "down"]):
        song_data = get_song_for_emotion(last_emotion)
        if song_data:
            print(f"Bot: {song_data['message']}")
            print(f"Bot: Here's a song matching your mood: '{song_data['song']['title']}' by {song_data['song']['artist']}")
            print(f"Listen here: {song_data['song']['url']}")
        else:
            print("Bot: I couldn't find a matching song, but I'm here if you want to talk more.")
        return True
    else:
        song_data = get_song_for_emotion(last_emotion)
        if song_data:
            print(f"Bot: I'll suggest a song that might fit your mood.")
            print(f"Listen to: '{song_data['song']['title']}' by {song_data['song']['artist']}")
            print(f"Listen here: {song_data['song']['url']}")
        else:
            print("Bot: I'm not sure what would help. Maybe try listening to your favorite song?")
        return True

def main():
    print("Bot: Hello! My name is Melody! I'm your mood-based music assistant. Share your feelings or ask me questions. Or type 'bye' to exit :( ")
    user_state = {
        "last_emotion": None,
        "awaiting_clarification": False
    }

    while True:
        try:
            user_input = input("You: ").strip()
            if not user_input:
                continue
            lower_input = user_input.lower()

            if any(w in lower_input for w in greetings_phrases):
                print("Bot: Hi! How are you feeling today?")
                continue

            if any(w in lower_input for w in how_are_you_phrases):
                print("Bot: I'm doing well, thanks for asking! How about you?")
                continue

            if any(w in lower_input for w in good_day_phrases):
                print("Bot: Nice to meet you! What music would suit your mood today?")
                continue

            if any(w in lower_input for w in farewell_phrases):
                print("Bot: Goodbye! Hope the music helped.")
                break

            if any(w in lower_input for w in thanks_phrases):
                print("Bot: You're very welcome! 😊")
                continue

            if any(w in lower_input for w in whats_up_phrases):
                print("Bot: Just here waiting to find you the perfect song! How are you feeling?")
                continue

            if any(w in lower_input for w in identity_phrases):
                print("Bot: I'm your personal mood-based music assistant! I suggest songs based on your emotions.")
                continue

            if any(w in lower_input for w in capability_phrases):
                print("Bot: I can: \n1. Detect your mood from your words\n2. Suggest matching songs\n3. Help improve your mood with music. Just let me know how you feel and i will help you! ♡ ")
                continue

            if "nevermind" in lower_input:
                print("Bot: No worries! If you want to talk later, I'm here.")
                continue

            if any(phrase in lower_input for phrase in offtopic_phrases):
                handle_offtopic_request()
                continue

            if any(phrase in lower_input for phrase in additional_phrases):
                print("Bot: Glad to help! How else can I help?")
                continue
            if user_state["awaiting_clarification"]:
                handled = handle_clarification_response(user_input, user_state["last_emotion"])
                user_state["awaiting_clarification"] = not handled
                continue
            result = identify_emotion_or_request(user_input)

            if result["type"] == "question":
                key = result["value"]
                if key == "who_are_you":
                    print("Bot: I'm your friendly mood-music assistant here to match your feelings with songs  ▶︎ •၊၊||၊|။|||||||• .")
                elif key == "what_can_you_do":
                    print("Bot: I listen to your feelings and suggest music to match or help improve your mood.")
                else:
                    print("Bot: Sorry, I don't understand that question.")
                continue

            elif result["type"] == "positive_emotion":
                emotion = result["value"]
                song_data = get_song_for_emotion(emotion)
                if song_data:
                    print(f"Bot: {random.choice(compliments)}")
                    print(f"Bot: {song_data['message']}")
                    print(f"Here's a song to celebrate your happiness: '{song_data['song']['title']}' by {song_data['song']['artist']}")
                    print(f"Listen here: {song_data['song']['url']}")
                else:
                    print(f"Bot: {random.choice(compliments)} Enjoy your happy mood!")
                user_state["last_emotion"] = emotion

            elif result["type"] == "negative_emotion":
                emotion = result["value"]
                user_state["last_emotion"] = emotion
                user_state["awaiting_clarification"] = True
                print(f"Bot: Would you like a song to help you feel better or to match your current mood of {emotion}? (,,>﹏<,,)")
                continue

            elif result["type"] == "unknown":
                print("Bot: I'm unable to help with that topic. But in the meantime, here's a song to lift your spirits. If you want me to help, please try to describe your emotions!' ")
                song_data = get_song_for_emotion("hope")
                if song_data:
                    print(f"Listen here: {song_data['song']['url']}")
                else:
                    print("Bot: Sorry, I couldn't find a song right now.")
        except Exception as e:
            print(f"Bot: Sorry, an error occurred: {str(e)}. Please try again.")

if __name__ == "__main__":
    main()

Bot: Hello! My name is Melody! I'm your mood-based music assistant. Share your feelings or ask me questions. Or type 'bye' to exit :( 
You: hello
Bot: Hi! How are you feeling today?
You: i feel upset
Bot: Would you like a song to help you feel better or to match your current mood of sadness? (,,>﹏<,,)
You: better
Bot: An empowering anthem about growing stronger through life's challenges.
Bot: I'm sending positive vibes your way! Try this uplifting song: 'Stronger (What Doesn't Kill You)' by Kelly Clarkson
Listen here: https://vk.com/audio-2001720841_3720841
You: wow
Bot: Glad to help! How else can I help?
You: what are you
Bot: I'm your personal mood-based music assistant! I suggest songs based on your emotions.
You: ae you a human
Bot: I'm unable to help with that topic. But in the meantime, here's a song to lift your spirits. If you want me to help, please try to describe your emotions!' 
Listen here: https://vk.com/audio-2001158098_34158098
You: are you a human
Bot: I'm your persona